In [71]:
import pandas as pd
from bs4 import BeautifulSoup
import regex
import re
import numpy
import requests
import pprint

In [72]:
def strip_word(word: str | None) -> str | None:
    if word == None:
        return None
    letters = []
    word = word.replace('[d]','')
    word = word.replace('[c]','')
    for i in word:
       if i.isalpha() or i.isspace():
            letters.append(i)
    word = ''.join(letters)
    word = word.strip()
    return word


In [73]:
def remove_banned(word: str, banned: list) -> str:
    translation_table = str.maketrans('', '', ''.join(banned))
    removed_char = word.translate(translation_table)
    print(removed_char)
    
         

In [74]:
def is_empty(word: str) -> bool:
    return word == ''
def transform_row(word: str) -> bool:
    if word == '—':
        return None
    else:
        return word

In [75]:
def ref_filter(word: str | None) -> str | None:
    result = ''
    if word == None:
        return None
    for letter in word:
        if letter.isalnum() or letter == '/':
            result += letter
        else:
            return result
    return result
            

In [76]:
url = 'https://en.wikipedia.org/wiki/Major_League_Baseball_Most_Valuable_Player_Award'
html = requests.get(url)


In [77]:
soup = BeautifulSoup(html.content, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [78]:
banned = ['American League winner','Year', 'Team', 'Position', 'National League winner']
tables = soup.select('.wikitable')[1:4]
col_names = []
data = {}
doube_row = False
table_num = 0
for table in tables:
    table_num += 1
    headers = table.select('tr th')
    #Below logic gets header fields
    for header in headers:
        stripped_header = header.text.strip()
        if header.get('scope') == 'col' and stripped_header not in data.keys() and stripped_header !='Ref':
            data[stripped_header] = []
    #Get all row values
    index = 0
    if table_num <= 2:
        for row in table.select('tbody tr td, th'):
            if row.get('title') != ''  and row.get('scope') != 'col' and row.get_text() != ''and row.get_text() != 'Ref' and row.get('role') != 'columnheader button':
                text = transform_row(row.text.strip())
                if index == 0:
                    data['Year'].append(text)
                    index += 1
                elif index == 1:
                    data['American League winner'].append(strip_word(text))
                    index += 1
                elif index == 2:
                    data['Team'].append(strip_word(text))
                    index += 1
                elif index == 3:
                    data['Position'].append(ref_filter(text))
                    index += 1
                elif index == 4:
                    data['National League winner'].append(strip_word(text))
                    index += 1
                elif index == 5:
                    data['Team'].append(strip_word(text))
                    index += 1
                elif index == 6:
                    index += 1
                    data['Position'].append(ref_filter(text))
                else:
                    index = 0
    else:
        for row in table.select('tbody tr td, th td'):
            if row.get('title') != ''  and row.get('scope') != 'col' and row.get_text() != ''and row.get_text() != 'Ref' and row.get('role') != 'columnheader button':
                text = transform_row(row.text.strip())
                #exception to account for a change in formatting of the table which upsets the index
                if row.text == 'Willie Stargell†[d]':
                    index = 4
                    data['Year'].append('1979')
                    data['American League winner'].append(None)
                    data['Position'].append(None)
                    data['Team'].append(None)
                if row.text == '1980':
                    index = 0
                if index == 0:
                    data['Year'].append(text)
                    index += 1
                elif index == 1:
                    data['American League winner'].append(strip_word(text))
                    index += 1
                elif index == 2:
                    data['Team'].append(strip_word(text))
                    index += 1
                elif index == 3:
                    data['Position'].append(ref_filter(text))
                    index += 1
                elif index == 4:
                    data['National League winner'].append(strip_word(text))
                    index += 1
                elif index == 5:
                    data['Team'].append(strip_word(text))
                    index += 1
                elif index == 6:
                    index += 1
                    data['Position'].append(ref_filter(text))
                else:
                    index = 0
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(data)

      

            
            
            
            


{ 'American League winner': [ 'Ty Cobb',
                              'Tris Speaker',
                              'Walter Johnson',
                              'Eddie Collins',
                              'George Sisler',
                              'Babe Ruth',
                              'Walter Johnson',
                              'Roger Peckinpaugh',
                              'George Burns',
                              'Lou Gehrig',
                              'Mickey Cochrane',
                              None,
                              'Lefty Grove',
                              'Jimmie Foxx',
                              'Jimmie Foxx',
                              'Mickey Cochrane',
                              'Hank Greenberg',
                              'Lou Gehrig',
                              'Charlie Gehringer',
                              'Jimmie Foxx',
                              'Joe DiMaggio',
                              'Hank 

In [79]:
def check_row_length(data_set: dict[str,list[str|None]]) -> None:
    for key in data_set.keys():
        key_length = len(data_set[key])
        print(f'{key} contains {key_length} items') 

In [80]:
check_row_length(data)

Year contains 104 items
American League winner contains 104 items
Team contains 208 items
Position contains 208 items
National League winner contains 104 items


In [81]:
for table in tables:
    for row in table.select('tbody tr td, th'):
        print(row.text)




Year

American League winner

Team

Position

National League winner

Team

Position

Ref

1911

Ty Cobb§

Detroit Tigers
OF

Frank Schulte

Chicago Cubs
OF
[19]

1912

Tris Speaker

Boston Red Sox
OF

Larry Doyle

New York Giants
2B
[20]

1913

Walter Johnson

Washington Senators
RHP

Jake Daubert

Brooklyn Superbas
1B
[21]

1914

Eddie Collins

Philadelphia Athletics
2B

Johnny Evers

Boston Braves
2B
[22]

Year

American League winner

Team

Position

National League winner

Team

Position

Ref

1922

George Sisler

St. Louis Browns
1B

—

—
—
[26]

1923

Babe Ruth§

New York Yankees
OF

—

—
—
[27]

1924

Walter Johnson (2)

Washington Senators
RHP

Dazzy Vance

Brooklyn Robins
RHP
[28]

1925

Roger Peckinpaugh

Washington Senators
SS

Rogers Hornsby

St. Louis Cardinals
2B
[29]

1926

George Burns

Cleveland Indians
1B

Bob O'Farrell

St. Louis Cardinals
C
[30]

1927

Lou Gehrig

New York Yankees
1B

Paul Waner

Pittsburgh Pirates
OF
[31]

1928

Mickey Cochrane

Philadelphia Athle

In [82]:
American_league_pos = []
National_league_pos = []
American_league_team = []
National_league_team = []
for i, pos in enumerate(data['Position']):
    if i%2 == 0:
        American_league_pos.append(pos)
    else:
        National_league_pos.append(pos)
for i, pos in enumerate(data['Team']):
    if i%2 == 0:
        American_league_team.append(pos)
    else:
        National_league_team.append(pos)
print(National_league_team)

['Chicago Cubs', 'New York Giants', 'Brooklyn Superbas', 'Boston Braves', None, None, 'Brooklyn Robins', 'St Louis Cardinals', 'St Louis Cardinals', 'Pittsburgh Pirates', 'St Louis Cardinals', 'Chicago Cubs', 'St Louis Cardinals', 'Philadelphia Phillies', 'New York Giants', 'St Louis Cardinals', 'Chicago Cubs', 'New York Giants', 'St Louis Cardinals', 'Cincinnati Reds', 'Cincinnati Reds', 'Cincinnati Reds', 'Brooklyn Dodgers', 'St Louis Cardinals', 'St Louis Cardinals', 'St Louis Cardinals', 'Chicago Cubs', 'St Louis Cardinals', 'Boston Braves', 'St Louis Cardinals', 'Brooklyn Dodgers', 'Philadelphia Phillies', 'Brooklyn Dodgers', 'Chicago Cubs', 'Brooklyn Dodgers', 'New York Giants', 'Brooklyn Dodgers', 'Brooklyn Dodgers', 'Milwaukee Braves', 'Chicago Cubs', 'Chicago Cubs', 'Pittsburgh Pirates', 'Cincinnati Reds', 'Los Angeles Dodgers', 'Los Angeles Dodgers', 'St Louis Cardinals', 'San Francisco Giants', 'Pittsburgh Pirates', 'St Louis Cardinals', 'St Louis Cardinals', 'San Francisco 

In [83]:
from hashlib import new


new_dict ={'Year':[],'American League winner':[],'American League Team':[], 'American League Position':[], 'National League winner':[], 'National League Team':[], 'National League Position':[]}
for key in data.keys():
    if key in new_dict.keys():
        new_dict[key] = data[key]
new_dict['American League Team'] = American_league_team
new_dict['National League Team'] = National_league_team
new_dict['National League Position'] = National_league_pos
new_dict['American League Position'] = American_league_pos

In [84]:
pp.pprint(new_dict)

{ 'American League Position': [ 'OF',
                                'OF',
                                'RHP',
                                '2B',
                                '1B',
                                'OF',
                                'RHP',
                                'SS',
                                '1B',
                                '1B',
                                'C',
                                None,
                                'LHP',
                                '1B',
                                '1B',
                                'C',
                                '1B',
                                '1B',
                                '2B',
                                '1B',
                                'OF',
                                'OF',
                                'OF',
                                '2B',
                                'RHP',
                                'LHP',
         

In [85]:
new_df = pd.DataFrame.from_dict(new_dict)
pd.set_option('display.max_row', None)

In [86]:
display(new_df)

,Year,American League winner,American League Team,American League Position,National League winner,National League Team,National League Position
0,1911,Ty Cobb,Detroit Tigers,OF,Frank Schulte,Chicago Cubs,OF
1,1912,Tris Speaker,Boston Red Sox,OF,Larry Doyle,New York Giants,2B
2,1913,Walter Johnson,Washington Senators,RHP,Jake Daubert,Brooklyn Superbas,1B
3,1914,Eddie Collins,Philadelphia Athletics,2B,Johnny Evers,Boston Braves,2B
4,1922,George Sisler,St Louis Browns,1B,None,None,None
5,1923,Babe Ruth,New York Yankees,OF,None,None,None
6,1924,Walter Johnson,Washington Senators,RHP,Dazzy Vance,Brooklyn Robins,RHP
7,1925,Roger Peckinpaugh,Washington Senators,SS,Rogers Hornsby,St Louis Cardinals,2B
8,1926,George Burns,Cleveland Indians,1B,Bob OFarrell,St Louis Cardinals,C
9,1927,Lou Gehrig,New York Yankees,1B,Paul Waner,Pittsburgh Pirates,OF


In [87]:
players_american = new_df['American League winner'].tolist()
players_national = new_df['National League winner'].tolist()
players = players_american + players_national
player_links = []
for table in tables:
    for i in table.find_all('a'):
        text = transform_row(i.text.strip())
        if strip_word(text) in players:
            player_links.append(i['href'])
        
         

In [88]:
players_american = new_df['American League winner'].tolist()
players_national = new_df['National League winner'].tolist()
players = players_american + players_national
print(players)

['Ty Cobb', 'Tris Speaker', 'Walter Johnson', 'Eddie Collins', 'George Sisler', 'Babe Ruth', 'Walter Johnson', 'Roger Peckinpaugh', 'George Burns', 'Lou Gehrig', 'Mickey Cochrane', None, 'Lefty Grove', 'Jimmie Foxx', 'Jimmie Foxx', 'Mickey Cochrane', 'Hank Greenberg', 'Lou Gehrig', 'Charlie Gehringer', 'Jimmie Foxx', 'Joe DiMaggio', 'Hank Greenberg', 'Joe DiMaggio', 'Joe Gordon', 'Spud Chandler', 'Hal Newhouser', 'Hal Newhouser', 'Ted Williams', 'Joe DiMaggio', 'Lou Boudreau', 'Ted Williams', 'Phil Rizzuto', 'Yogi Berra', 'Bobby Shantz', 'Al Rosen', 'Yogi Berra', 'Yogi Berra', 'Mickey Mantle', 'Mickey Mantle', 'Jackie Jensen', 'Nellie Fox', 'Roger Maris', 'Roger Maris', 'Mickey Mantle', 'Elston Howard', 'Brooks Robinson', 'Zoilo Versalles', 'Frank Robinson', 'Carl Yastrzemski', 'Denny McLain', 'Harmon Killebrew', 'Boog Powell', 'Vida Blue', 'Dick Allen', 'Reggie Jackson', 'Jeff Burroughs', 'Fred Lynn', 'Thurman Munson', 'Rod Carew', 'Jim Rice', 'Don Baylor', None, 'George Brett', 'Roll

In [94]:
print((player_links))

['/wiki/Ty_Cobb', '/wiki/Frank_Schulte', '/wiki/Tris_Speaker', '/wiki/Larry_Doyle_(baseball)', '/wiki/Walter_Johnson', '/wiki/Jake_Daubert', '/wiki/Eddie_Collins', '/wiki/Johnny_Evers', '/wiki/George_Sisler', '/wiki/Babe_Ruth', '/wiki/Walter_Johnson', '/wiki/Dazzy_Vance', '/wiki/Roger_Peckinpaugh', '/wiki/Rogers_Hornsby', '/wiki/George_Burns_(first_baseman)', '/wiki/Bob_O%27Farrell', '/wiki/Lou_Gehrig', '/wiki/Paul_Waner', '/wiki/Mickey_Cochrane', '/wiki/Jim_Bottomley', '/wiki/Rogers_Hornsby', '/wiki/Lefty_Grove', '/wiki/Frankie_Frisch', '/wiki/Jimmie_Foxx', '/wiki/Chuck_Klein', '/wiki/Jimmie_Foxx', '/wiki/Carl_Hubbell', '/wiki/Mickey_Cochrane', '/wiki/Dizzy_Dean', '/wiki/Hank_Greenberg', '/wiki/Gabby_Hartnett', '/wiki/Lou_Gehrig', '/wiki/Carl_Hubbell', '/wiki/Charlie_Gehringer', '/wiki/Joe_Medwick', '/wiki/Jimmie_Foxx', '/wiki/Ernie_Lombardi', '/wiki/Joe_DiMaggio', '/wiki/Bucky_Walters', '/wiki/Hank_Greenberg', '/wiki/Frank_McCormick', '/wiki/Joe_DiMaggio', '/wiki/Dolph_Camilli', '/wi

In [90]:
print(players)

['Ty Cobb', 'Tris Speaker', 'Walter Johnson', 'Eddie Collins', 'George Sisler', 'Babe Ruth', 'Walter Johnson', 'Roger Peckinpaugh', 'George Burns', 'Lou Gehrig', 'Mickey Cochrane', None, 'Lefty Grove', 'Jimmie Foxx', 'Jimmie Foxx', 'Mickey Cochrane', 'Hank Greenberg', 'Lou Gehrig', 'Charlie Gehringer', 'Jimmie Foxx', 'Joe DiMaggio', 'Hank Greenberg', 'Joe DiMaggio', 'Joe Gordon', 'Spud Chandler', 'Hal Newhouser', 'Hal Newhouser', 'Ted Williams', 'Joe DiMaggio', 'Lou Boudreau', 'Ted Williams', 'Phil Rizzuto', 'Yogi Berra', 'Bobby Shantz', 'Al Rosen', 'Yogi Berra', 'Yogi Berra', 'Mickey Mantle', 'Mickey Mantle', 'Jackie Jensen', 'Nellie Fox', 'Roger Maris', 'Roger Maris', 'Mickey Mantle', 'Elston Howard', 'Brooks Robinson', 'Zoilo Versalles', 'Frank Robinson', 'Carl Yastrzemski', 'Denny McLain', 'Harmon Killebrew', 'Boog Powell', 'Vida Blue', 'Dick Allen', 'Reggie Jackson', 'Jeff Burroughs', 'Fred Lynn', 'Thurman Munson', 'Rod Carew', 'Jim Rice', 'Don Baylor', None, 'George Brett', 'Roll

In [97]:
player_url = 'https://en.wikipedia.org/wiki/Ty_Cobb'
player_html = requests.get(player_url)
player_soup = BeautifulSoup(player_html.content, 'html.parser')

In [105]:
#player_table_info = player_soup.select('table tbody th, td')
#for i in player_table_info:
   # if i.text == 'MLB statistics':
        

<th class="infobox-header" colspan="2">MLB statistics</th>


In [111]:
start = player_soup.find('th', text = 'MLB statistics').parent
print(start)

<tr style="background-color:#DCDCDC;color:#000000;"><th class="infobox-header" colspan="2">MLB statistics</th></tr>


In [122]:
for tr in start.find_next_siblings('tr'):
    if tr.find('th', text = 'Teams'):
        break
    print(tr.find('a').text)
    print(tr.find('td').text)
    
    

Batting average
.367
Hits
4,191
Home runs
117
Runs batted in
1,938
Stolen bases
892


AttributeError: 'NoneType' object has no attribute 'text'